# Play Tic-Tac-Toe Interactively (Simple Implementation)

## The board

I represent the board as a vector of length 9. The values are `' ', 'x', 'o'`.  

In [1]:
def empty_board():
    return [' '] * 9

board = empty_board()
display(board)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

### Helper functions

Show the board.

In [2]:
import numpy as np

def show_board(board):
    """display the board"""
    b = np.array(board).reshape((3,3))
    print(b)

board = empty_board()
show_board(board)    

print()
print("Add some x's")
board[0] = 'x'; board[3] = 'x'; board[6] = 'x';  
show_board(board)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Add some x's
[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


Determine if the current board/state has a winner.

In [3]:
def check_win(board):
    """check the board and return one of x, o, d (draw), or n (for next move)"""

    board = np.array(board).reshape((3,3))
   
    diagonals = np.array([[board[i][i] for i in range(len(board))], 
                          [board[i][len(board)-i-1] for i in range(len(board))]])
    
    for a_board in [board, np.transpose(board), diagonals]:
        for row in a_board:
            if len(set(row)) == 1 and row[0] != ' ':
                return row[0]

    # check for draw
    if(np.sum(board == ' ') < 1):
        return 'd'
    
    return 'n'

show_board(board)
print('Win? ' + check_win(board))

print()
show_board(empty_board())
print('Win? ' + check_win(empty_board()))

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]
Win? x

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
Win? n


What are the possible actions given the current board?

In [4]:
def actions(board):
    """return possible actions as a vector of indices"""
    return np.where(np.array(board) == ' ')[0].tolist()

show_board(board)
actions(board)

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


[1, 2, 4, 5, 7, 8]

What is the new state after executing an action. 

In [5]:
def result(state, player, action):
    """Add move to the board."""
    
    state = state.copy()
    state[action] = player
  
    return state

show_board(empty_board())

print()
print("State for placing an x at position 4:")
show_board(result(empty_board(), 'x', 4))

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

State for placing an x at position 4:
[[' ' ' ' ' ']
 [' ' 'x' ' ']
 [' ' ' ' ' ']]


## Experiments


### Baseline: Randomized Player

A completely randomized player agent can be used as a weak baseline.

In [6]:
def random_player(board, player = None):
    """Simple player that chooses a random empy square (equal probability of all permissible actions). 
    player is unused."""
    return np.random.choice(actions(board))

show_board(board)
%time random_player(board)

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]
CPU times: user 394 µs, sys: 192 µs, total: 586 µs
Wall time: 850 µs


5

### The Environment

Implement the environment that calls the agent. The percept is the board and the action is move.

In [72]:
def switch_player(player, x, o):
    if player == 'x':
        return 'o', o
    else:
        return 'x', x

def play(x, o, N = 1, show_final_board = False):
    """Let two agents play each other N times. x starts. x and y are agent functions that 
    get the board as the percept and return their next action."""
    results = {'x': 0, 'o': 0, 'd': 0}
    
    for i in range(N):
        board = empty_board()
        player, fun = 'x', x
        
        while True:
            a = fun(board, player)
            board = result(board, player, a)
            
            win = check_win(board)   # returns the 'n' if the game is not done.
            if win != 'n':
                results[win] += 1
                if show_final_board: 
                    show_board(board)
                break
            
            player, fun = switch_player(player, x, o)   
    
    return results

In [74]:
play(random_player, random_player, show_final_board=True)

[['o' 'x' 'o']
 ['o' 'x' 'o']
 ['x' 'x' 'x']]


{'x': 1, 'o': 0, 'd': 0}

Implement the interactive player. It shows the board and asks you for the move.

In [64]:
from IPython.display import clear_output

def interactive_player(board, player = None):
    clear_output(wait = False)
    show_board(board)
    
    available = actions(board)
    print(f'Available actions are: {available}')

    retry = True
    while retry:
        try:
            move = int(input("Your move:\n"))
            if move in available:
                retry = False
            else:
                raise ValueError()
        except ValueError:
            print("Please enter a valid move.")

    return(move)

Start an interactive game as x

In [75]:
play(interactive_player, random_player, show_final_board=True)

[['o' 'x' ' ']
 [' ' 'x' ' ']
 ['o' ' ' ' ']]
Available actions are: [2, 3, 5, 7, 8]


Your move:
 7


[['o' 'x' ' ']
 [' ' 'x' ' ']
 ['o' 'x' ' ']]


{'x': 1, 'o': 0, 'd': 0}

Start an interactive game as o

In [76]:
play(random_player, interactive_player, show_final_board=True)

[['o' 'o' ' ']
 ['x' 'o' 'x']
 ['x' ' ' 'x']]
Available actions are: [2, 7]


Your move:
 3


Please enter a valid move.


Your move:
 2


[['o' 'o' 'o']
 ['x' 'o' 'x']
 ['x' ' ' 'x']]


{'x': 0, 'o': 1, 'd': 0}